# Compressing images in the YCoCg domain

Compare the performance of compressing images in the RGB and YCoCg domains.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pylab
%matplotlib inline
import image_3 as image
import image_1 as component
import YCoCg as YUV
import deadzone as Q
import distortion
import information
import math

## Global parameters of the notebook

In [ ]:
#test_image = "../sequences/stockholm/"
test_image = "../sequences/lena_color/"
#test_image = "../sequences/lena_bw/"

In [ ]:
N_components = 3

## RGB components information

In [ ]:
x = image.read(test_image, 0)
print("component maximum mininum d-range average std-dev entropy        energy  avg-enegy")
accumulated_entropy = 0
for _c in range(N_components):
    comp = x[..., _c]
    max = comp.max()
    min = comp.min()
    d_range = max - min
    entropy = information.entropy(comp.flatten())
    accumulated_entropy += entropy
    print(f"{_c:9d} {max:7.1f} {min:7.1f} {d_range:7.1f} {np.average(comp):7.1f} {math.sqrt(np.var(comp)):7.1f} {entropy:7.1f} {information.energy(comp):13.1f} {information.energy(comp)/comp.size:10.1f}")
avg_entropy = accumulated_entropy / x.shape[2]
print("Average entropy by RGB components:", avg_entropy)
print("Entropy of the image in the RGB domain:", information.entropy(x.flatten()))

Notice that the average entropy of the image by components and the entropy of the image can be different because the probabilities of the components are different. This, for example, indicates that it could be better to compress the image by compressing each component independly than to compress the image considering all the components together.

## YCoCg components information

In [ ]:
x = image.read(test_image, 0)
xx = YUV.from_RGB(x.astype(np.int16))
print("component maximum mininum max-min average std-dev entropy        energy  avg-enegy")
accumulated_entropy = 0
for _c in range(N_components):
    comp = xx[..., _c]
    max = comp.max()
    min = comp.min()
    d_range = max - min
    entropy = information.entropy(comp.flatten())
    accumulated_entropy += entropy
    print(f"{_c:9d} {max:7.1f} {min:7.1f} {d_range:7.1f} {np.average(comp):7.1f} {math.sqrt(np.var(comp)):7.1f} {entropy:7.1f} {information.energy(comp):13.1f} {information.energy(comp)/comp.size:10.1f}")
avg_entropy = accumulated_entropy / xx.shape[2]
print("Average entropy by YCoCg components:", avg_entropy)
print("Entropy of the image in the YCoCg domain:", information.entropy(xx.flatten()))

In [ ]:
x = np.full_like(x, fill_value=255)
xx = YUV.from_RGB(x.astype(np.int16))
print("component maximum mininum max-min average std-dev entropy        energy  avg-enegy")
accumulated_entropy = 0
for _c in range(N_components):
    comp = xx[..., _c]
    max = comp.max()
    min = comp.min()
    d_range = max - min
    entropy = information.entropy(comp.flatten())
    accumulated_entropy += entropy
    print(f"{_c:9d} {max:7.1f} {min:7.1f} {d_range:7.1f} {np.average(comp):7.1f} {math.sqrt(np.var(comp)):7.1f} {entropy:7.1f} {information.energy(comp):13.1f} {information.energy(comp)/comp.size:10.1f}")
avg_entropy = accumulated_entropy / xx.shape[2]
print("Average entropy by YCoCg components:", avg_entropy)
print("Entropy of the image in the YCoCg domain:", information.entropy(xx.flatten()))

In [ ]:
x = np.full_like(x, fill_value=255)
x[1::2,::2] = 0
x[::2,1::2] = 0
image.show(x[:10, :10], "A chess matrix (top-left corner detail)")
xx = YUV.from_RGB(x.astype(np.int16))
print("component maximum mininum d-range average std-dev entropy        energy  avg-enegy")
accumulated_entropy = 0
for _c in range(N_components):
    comp = xx[..., _c]
    max = comp.max()
    min = comp.min()
    d_range = max - min
    entropy = information.entropy(comp.flatten())
    accumulated_entropy += entropy
    print(f"{_c:9d} {max:7.1f} {min:7.1f} {d_range:7.1f} {np.average(comp):7.1f} {math.sqrt(np.var(comp)):7.1f} {entropy:7.1f} {information.energy(comp):13.1f} {information.energy(comp)/comp.size:10.1f}")
avg_entropy = accumulated_entropy / xx.shape[2]
print("Average entropy by YCoCg components:", avg_entropy)
print("Entropy of the image in the YCoCg domain:", information.entropy(xx.flatten()))

We can see that:
1. The YCoCg transform reduces the entropy, globally and by components.
2. The YCoCg transform accumulates most of the energy in the Y component.
3. The Y component needs the same number of bits (eight) than the R, G and B components. However, if you see the code you can also deduce this.
4. The Co and the Cg components can be negative.

## Defining the quantization steps

This parameter should be defined depending on the dynamic range of que quantization indexes and the dynamic range of the entropy encoder (image.write()) that we want to use. In our case, both color domains (RGB and YCoCg) need 8 bits, and in general, this will increase the compression ratio of the entropy encoder. When we need more than 8 bits, the next "step" is 16 bits, which is less efficient.

In [ ]:
Q_steps = [128, 64, 32, 16, 8, 4, 2, 1]

## RD curve in the RGB domain

In [ ]:
x = image.read(test_image, 0)

RGB_points = []
for Q_step in Q_steps:
    x_k = Q.quantize(x, Q_step)
    x_dQ = Q.dequantize(x_k, Q_step)
    BPP = image.write(x_k.astype(np.uint8), f"/tmp/RGB_{Q_step}_", 0)*8/x.size
    # Remember that image.write() has been designed to output positive integer data.
    __ = image.read(f"/tmp/RGB_{Q_step}_", 0)
    assert (x_k == __).all()
    MSE = distortion.MSE(x, x_dQ)
    point = (BPP, MSE)
    print(point)
    RGB_points.append(point)

In [ ]:
# For future references
with open('RGB.txt', 'w') as f:
    for item in RGB_points:
        f.write(f"{item[0]}\t{item[1]}\n")

## RD curve in the YUV domain

In [ ]:
x = image.read(test_image, 0)
xx = YUV.from_RGB(x.astype(np.int16))

YUV_points = []
for Q_step in Q_steps:
    xx_k = Q.quantize(xx, Q_step)
    xx_dQ = Q.dequantize(xx_k, Q_step)
    print(xx_k.dtype, xx_k.max(), xx_k.min())
    #BPP = image.write((xx_k.astype(np.int32) + 32768).astype(np.uint16), f"/tmp/YUV_{Q_step}_", 0)*8/x.size
    #__ = image.read(f"/tmp/YUV_{Q_step}_", 0).astype(np.int32) - 32768
    #BPP = image.write((xx_k.astype(np.int16) + 128).astype(np.uint8), f"/tmp/YUV_{Q_step}_", 0)*8/x.size
    #__ = image.read(f"/tmp/YUV_{Q_step}_", 0).astype(np.int32) - 128
    #BPP = image.write((xx_k + 128).astype(np.uint8), f"/tmp/YUV_{Q_step}_", 0)*8/x.size
    #__ = image.read(f"/tmp/YUV_{Q_step}_", 0).astype(np.int16) - 128
    xx_k[..., 1] += 128
    xx_k[..., 2] += 128
    BPP = image.write(xx_k.astype(np.uint8), f"/tmp/YCoCg_{Q_step}_", 0)*8/xx.size
    __ = image.read(f"/tmp/YCoCg_{Q_step}_", 0)
    #BPP = image.write(xx_k, f"/tmp/YUV_{Q_step}_", 0)*8/x.size
    #BPP = image.write((xx_k + 128).astype(np.uint8), f"/tmp/YUV_{Q_step}_", 0)*8/x.size
    #BPP = image.write(xx_k + xx_k.min(), f"/tmp/YUV_{Q_step}_", 0)*8/x.size
    #BPP = image.write(xx_k - xx.min(), f"/tmp/YUV_{Q_step}_", 0)*8/x.size
    #BPP = image.write(xx_k + 256, f"/tmp/YUV_{Q_step}_", 0)*8/x.size
    #for i in range(512):
    #    for j in range(512):
    #        if (xx_k[i,j] != __[i,j]).any():
    #            print(Q_step, i, j, x_k[i,j], __[i,j])
    #            break
    try:
        assert (xx_k == __).all()
    except AssertionError:
        counter = 0
        for _i in range(x.shape[0]):
            for _j in range(x.shape[1]):
                if (xx_k[_i, _j] != __[_i, _j]).any():
                    print(xx_k[_i, _j], __[_i, _j])
                    if counter > 10:
                        break
                    counter += 1
            if counter > 10:
                break
    x_dQ = YUV.to_RGB(xx_dQ)
    MSE = distortion.MSE(x, x_dQ)
    point = (BPP, MSE)
    print(point)
    YUV_points.append(point)

In [ ]:
# For ...
with open('YCoCg.txt', 'w') as f:
    for item in YUV_points:
        f.write(f"{item[0]}\t{item[1]}\n")

## Compare

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RGB_points), label="RGB")
pylab.plot(*zip(*YUV_points), label="YUV")
pylab.title("Which color domain is better?")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc="best")
pylab.show()

The YCoCg domain seems to be better than the RGB one at low bit-rates (low quality), but slightly worse at medium bit-rates (intermediate quality). For this reason, the YCoCg color transform will be used in the rest of experiments.

Please, confirm this for other images. Plot the average curve considering also the standard deviation.

## Let's optimize the quantization step by quantizing each component indepently

The performance of any image compressor depends on how much of the original information can be stored in the code-stream. In other words, the performance is proportional to the amount of distortion accumulated in the decoded image divided by the length of the code-stream (notice that the lower the distortion, the higher the performance, and the lower the length, the higher the performance, again). This is exactly what an *operational* RD curve of the image represents.

In the previous experiment we have used the same quantization step for the three color components. However, although this is the fastest quantization strategy, not necessaryly have to be the optimal one from a RD perpective because the contribution of the components to the reconstructed image can be different.

An important question that arises here is: are the contributions of the components to the quality of the reconstructed image independent? If the transform is orthogonal (something that we know that if false because the channel Y and Cg share the green RGB information and therefore, are correlated), the answer would be yes. Under the umbrella of orthogonality, we can assume that the quantization error generated in one component does not influence on the quantization error added to the other components, and therefore, we can optimize (search) the quantization step in each component, independently, without computing the distortion in the RGB image domain.

The YCoCg is not an orthogonal transform, but it can be considered near-orthogonal. Therefore, let's compute the RD contribution (a slope of one step in the RD curve) of each component for each quantization step, and define a quantization algoritm in which we select progressively smaller contributions, starting at the higher one. We will also compare the performance of measuring the distortion in the YCoCg and the RGB domains.

Algorithm:
1. Read the image.
2. Transform to the YCoCg domain.
3. Find a RD curve for each component.
4. Compute the slope of each step of each curve and put all the slopes in the same list.
5. Sort the previous list by the slope field.
6. Find the RD curve the progressively uses descending slopes.

In [ ]:
# Read the image and move to the YCoCg domain.
x = image.read(test_image, 0)
xx = YUV.from_RGB(x.astype(np.int16))

# The croma is moved to a positive domain
xx[..., 1] += 128
xx[..., 2] += 128

In [ ]:
for _c in range(N_components):
    print(xx[...,_c].max(), xx[...,_c].min())

In [ ]:
# Create a list of RD points for each RD curve (component) and the corresponding lists of RD slopes between these points.
# The first coordinate of each point is the rate and the second coordinate is the corresponding distortion.
# The first point of each RD curve has a distortion equal to the energy of the component and a rate=0.
# Notice that no slope can be computed for the first point.
RD_points = []  # MSEs in the YCoCg domain
RD_slopes = []
RD_points2 = [] # MSEs in the RGB domain
RD_slopes2 = []
for _c in range(N_components):
    comp = xx[..., _c]
    comp_energy = information.energy(comp)
    RD_points.append([(0, comp_energy)])
    RD_slopes.append([])
    RD_points2.append([(0, comp_energy)])
    RD_slopes2.append([])

In [ ]:
for _i,_j in enumerate(RD_points):
    print(_i,_j)
    
for _i,_j in enumerate(RD_slopes):
    print(_i,_j)

In [ ]:
# Populate the rest of points of each curve (component).
# We estimate the distortion in the YCoCg domain.
for _c in range(N_components):
    comp = xx[..., _c]
    Q_step_number = 0
    for Q_step in Q_steps:
        comp_k = Q.quantize(comp, Q_step)
        comp_dQ = Q.dequantize(comp_k, Q_step)
        MSE = distortion.MSE(comp, comp_dQ)
        BPP = component.write(comp_k.astype(np.uint8), f"/tmp/{_c}_{Q_step}_", 0)*8/xx.size
        __ = component.read(f"/tmp/{_c}_{Q_step}_", 0)
        try:
            assert (comp_k == __).all()
        except AssertionError:
            counter = 0
            for _i in range(x.shape[0]):
                for _j in range(x.shape[1]):
                    if (xx_k[_i, _j] != __[_i, _j]).any():
                        print(xx_k[_i, _j], __[_i, _j])
                        if counter > 10:
                            break
                        counter += 1
                if counter > 10:
                    break
        point = (BPP, MSE)
        print("Q_step =", Q_step, "BPP =", point[0], "MSE =", point[1])
        RD_points[_c].append(point)
        delta_BPP = BPP - RD_points[_c][Q_step_number][0]
        delta_MSE = RD_points[_c][Q_step_number][1] - MSE
        if delta_BPP > 0:
            slope = delta_MSE/delta_BPP
        else:
            slope = 0
        RD_slopes[_c].append((Q_step, slope, _c))
        Q_step_number += 1

In [ ]:
# In this version of the previous cell we measure the distortion in the RGB domain.
# For this, after quantizing a YCoCg component we compute the inverse YCoCg
# transform with the other components without quantization.
for _c in range(N_components):
    Q_step_number = 0
    for Q_step in Q_steps:
        xx_ = xx.copy()
        comp_k = Q.quantize(xx[..., _c], Q_step)
        BPP = component.write(comp_k.astype(np.uint8), f"/tmp/{_c}_{Q_step}_", 0)*8/xx.size
        __ = component.read(f"/tmp/{_c}_{Q_step}_", 0)
        try:
            assert (comp_k == __).all()
        except AssertionError:
            counter = 0
            for _i in range(x.shape[0]):
                for _j in range(x.shape[1]):
                    if (comp_k[_i, _j] != __[_i, _j]).any():
                        print("------->", comp_k[_i, _j], __[_i, _j])
                        if counter > 10:
                            break
                        counter += 1
                if counter > 10:
                    break
        xx_[..., _c] = Q.dequantize(comp_k, Q_step)
        xx_[..., 1] -= 128
        xx_[..., 2] -= 128
        MSE = distortion.MSE(x, YUV.to_RGB(xx_))
        point = (BPP, MSE)
        print("Q_step =", Q_step, "BPP =", point[0], "MSE =", point[1])
        RD_points2[_c].append(point)
        delta_BPP = BPP - RD_points2[_c][Q_step_number][0]
        delta_MSE = RD_points2[_c][Q_step_number][1] - MSE
        if delta_BPP > 0:
            slope = delta_MSE/delta_BPP
        else:
            slope = 0
        RD_slopes2[_c].append((Q_step, slope, _c))
        Q_step_number += 1

In [ ]:
print(RD_slopes2)

In [ ]:
# Show the slopes
RD_slopes_without_sb_index = []
for _c in range(N_components):
    RD_slopes_without_sb_index.append([])
for _c in range(N_components):
    for Q_step in range(len(Q_steps)):
        RD_slopes_without_sb_index[_c].append(RD_slopes[_c][Q_step][0:2])

RD_slopes_without_sb_index2 = []
for _c in range(N_components):
    RD_slopes_without_sb_index2.append([])
for _c in range(N_components):
    for Q_step in range(len(Q_steps)):
        RD_slopes_without_sb_index2[_c].append(RD_slopes2[_c][Q_step][0:2])

pylab.figure(dpi=150)
for _c in range(N_components):
    pylab.plot(*zip(*RD_slopes_without_sb_index[_c]), label=f"YCoCg {_c}", marker=0)
    pylab.plot(*zip(*RD_slopes_without_sb_index2[_c]), label=f"RGB {_c}", marker=1)
pylab.title("Slopes of the RD curves of the components")
pylab.xlabel("Q_step")
pylab.ylabel("Slope")
plt.legend(loc="best")
pylab.show()

In [ ]:
# Sort the slopes
single_list = []
single_list2 = []
for _c in range(N_components):
    for Q_step in range(len(Q_steps)):
        single_list.append(tuple(RD_slopes[_c][Q_step]))
        single_list2.append(tuple(RD_slopes2[_c][Q_step]))
sorted_slopes = sorted(single_list, key=lambda x: x[1])[::-1]
sorted_slopes2 = sorted(single_list2, key=lambda x: x[1])[::-1]

In [ ]:
print("MSE in YUV domain\t\t MSE in RGB domain")
for _i, _j in zip(sorted_slopes, sorted_slopes2):
    print(_i, '\t', _j)

In [ ]:
def quantize(x, Q_steps):
    x_k = np.empty_like(x)
    for _c in range(x.shape[2]):
        x_k[..., _c] = Q.quantize(x[..., _c], Q_steps[_c])
    return x_k

def dequantize(x_k, Q_steps):
    x_dQ = np.empty_like(x_k)
    for _c in range(x.shape[2]):
        x_dQ[..., _c] = Q.dequantize(x_k[..., _c], Q_steps[_c])
    return x_dQ

In [ ]:
# Find the optimal RD curve
optimal_RD_points = []
zz = np.zeros_like(xx)
Q_steps_combination = np.full(shape=(3,), fill_value=99999999)
for s in sorted_slopes:
    component_number = s[2]
    Q_steps_combination[component_number] = s[0]
    print(component_number, Q_steps_combination[component_number])
    zz[..., component_number] = xx[..., component_number]
    zz_k = quantize(zz, Q_steps_combination)
    zz_dQ = dequantize(zz_k, Q_steps_combination)
    z_dQ = YUV.to_RGB(zz_dQ)
    # If the color transform domain is not linear, the MSE should be measured in the RGB domain
    MSE = distortion.MSE(xx, zz_dQ)
    BPP = image.write((zz_k + 128).astype(np.uint8), f"/tmp/{component_number}_{Q_step}_", 0)*8/x.size
    point = (BPP, MSE)
    print("Q_step =", s[0], "BPP =", BPP, "MSE =", MSE)
    optimal_RD_points.append(point)

In [ ]:
optimal_RD_points2 = []
zz = np.zeros_like(xx)
Q_steps_combination = np.full(shape=(3,), fill_value=99999999)
for s in sorted_slopes2:
    component_number = s[2]
    Q_steps_combination[component_number] = s[0]
    print(component_number, Q_steps_combination[component_number])
    zz[..., component_number] = xx[..., component_number]
    zz_k = quantize(zz, Q_steps_combination)
    zz_dQ = dequantize(zz_k, Q_steps_combination)
    z_dQ = YUV.to_RGB(zz_dQ)
    # If the color transform domain is not linear, the MSE should be measured in the RGB domain
    MSE = distortion.MSE(xx, zz_dQ)
    BPP = image.write((zz_k + 128).astype(np.uint8), f"/tmp/{component_number}_{Q_step}_", 0)*8/x.size
    optimal_RD_points2.append((BPP, MSE))

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*YUV_points), label="YCoCg Constant quantization")
pylab.plot(*zip(*optimal_RD_points), label="YCoCg Optimal quantization (MSE in YCoCg domain)")
pylab.plot(*zip(*optimal_RD_points2), label="YCoCg Optimal quantization (MSE in RGB domain)")
pylab.title("RD optimization in the YUV domain")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc="best")
pylab.show()

The RD curves are not identical, which means that the YCoCg is not orthogonal, but, the results are quite close. Moreover, at least for this image, the performance of measuring the distortion in the YCoCg domain is slighly better than the performance of measuring the distortion in the RGB domain. 

Please, confirm this findings for other images. Plot the average curve considering also the standard deviation.